# Comparison Plot Demo

In [ ]:
import sys
sys.path.append('../../../../estimagic/')
from collections import namedtuple
from estimagic.visualization.comparison_plot import comparison_plot
import pandas as pd
import numpy as np
from numpy.random import uniform
np.random.seed(89345)

In [ ]:
from bokeh.plotting import output_notebook
output_notebook()

## Functions for creating artificial estimagic optimization results lists

In [ ]:
BLUEPRINT = namedtuple('optimization_result', ['params', 'info'])

def create_comparison_plot_inputs(model_class_dict):
    results_list = []
    counter = 0
    for model_class, specs in model_class_dict.items():
        for m in range(specs['nr_models']): 
            df = create_result_df(specs)
            
            info = {
                'model_name': 'mod_{}'.format(counter),
                'model_class': model_class
            }
            results_list.append(BLUEPRINT(df, info))
            counter += 1
    return results_list


def create_result_df(specs):
    df = pd.DataFrame()
    group_col = [] 
    final_values = []
    ci_lower = []
    ci_upper = []
    names = []
    for name, nr_params in specs['param_tuples']:
        names += ['{}_{}'.format(name, i) for i in range(nr_params)]
        group_col += [name] * nr_params
        mean = np.random.randint(-5, 10)
        std = np.random.uniform(0, 4)
        vals = np.random.normal(mean, std, nr_params)
        final_values += vals.tolist()
        ci_lower += (vals - uniform(- 0.2, std)).tolist()
        ci_upper += (vals + uniform(+ 0.2, std)).tolist()
    df['group'] = group_col
    df['name'] = names
    df['value'] = final_values
    df['conf_int_lower'] = ci_lower
    df['conf_int_upper'] = ci_upper
    return df

## Generate some artifical results

In [ ]:
models1 = {
    'small': {
        'nr_models': 15, 
        'param_tuples': [('covariance', 2), ('wage reg slope', 1)]
    },
    'large': {
        'nr_models': 14, 
        'param_tuples': [('covariance', 2), ('wage reg slope', 1)]
    },
    'other': {
        'nr_models': 10, 
        'param_tuples': [('covariance', 2)]
    },
}

reslist1 = create_comparison_plot_inputs(models1)

In [ ]:
color_dict={
    'small': 'goldenrod',
    'large': 'navy',
    'other': 'firebrick', 
}

## Generate the comparison plot

In [ ]:
df, grid = comparison_plot(reslist1, color_dict)

### Seaborn Stripplot as Comparison

In [ ]:
from estimagic.visualization.comparison_plot_data_preparation import (
    _consolidate_parameter_attribute,
    _combine_params_data
)
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
# generate data in the right format for seaborn
parameter_groups = _consolidate_parameter_attribute(
    reslist1, 'group')
parameter_names = _consolidate_parameter_attribute(
    reslist1, 'name')
all_data = _combine_params_data(
    results=reslist1, 
    parameter_groups=parameter_groups,
    parameter_names=parameter_names,
    color_dict=color_dict)

meas_vars = all_data[all_data['group'] == 'wage reg slope']

In [ ]:
hue_order = ['large', 'other', 'small']
sns_colors = sns.color_palette([
    color_dict[model_class] for model_class in hue_order])

sns.stripplot(
    x="value", y="name", data=meas_vars, hue='model_class', 
    palette=sns_colors, hue_order=hue_order)